# Citation Edgelist ROR to ROR

In [5]:
import pandas as pd
import numpy as np
import sys
from collections import Counter
from os import path
from pyate import combo_basic
from math import log, e
import math

# N.B., need to install dfply ----
from dfply import *


In [6]:
###############################
### Functions
###############################

def normalize_counter(x):
	total = sum(x.values(), 0.0)
	for key in x:
		x[key] /= total
	return x

In [7]:
###############################
### Input Parameters
###############################

# Do we include all authors or just the first/last author -------
input_author_position = "AllAuthors"

if input_author_position == "AuthorFirstLast":
	remove_position_ = "middle"
else:
	remove_position_ = "ANYTHING"

In [8]:
###############################
### Read in File
###############################

being_cited_df = pd.read_csv("INPUT_SQL_Citation_Data_Astronomy_and_Astrophysics.csv")

ror_df = pd.read_csv("INPUT_ROR_Metadata.csv")

In [9]:
###############################
### Prepare File
###############################

being_cited_df["cited_publication_year"] = being_cited_df["cited_publication_year"].astype(int)

year_list = being_cited_year_list = being_cited_df.cited_publication_year.drop_duplicates().tolist()
year_list.sort()

being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_dict('list') for year_ in being_cited_year_list}


/tmp/ipykernel_61538/358318943.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_dict('list') for year_ in being_cited_year_list}
/tmp/ipykernel_61538/358318943.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_dict('list') for year_ in being_cited_year_list}
/tmp/ipykernel_61538/358318943.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_di

/tmp/ipykernel_61538/358318943.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_dict('list') for year_ in being_cited_year_list}
/tmp/ipykernel_61538/358318943.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  being_cited_dict = {year_:being_cited_df.query("cited_publication_year==@year_")[['cited_work_id','cited_ror_paper','citing_paper']].drop_duplicates().set_index('cited_work_id').T.to_dict('list') for year_ in being_cited_year_list}


In [16]:
###############################
### Loop through each year in discipline
###############################

citation_list = []
full_citation_df = pd.DataFrame()

# Loop through each year ---------
for input_window in [2,5,10]:
	
	updated_year_list = [year_ for year_ in year_list  if (year_+input_window<(2022-input_window))]

	for year in year_list:	

			being_cited_year_ = {article_id: article_ for article_id, article_ in being_cited_dict[year].items()}

			for article_id, article_ in being_cited_year_.items():

				if (pd.isna(article_[1])):
					continue

				# N.B., Articles need to have received at least two citations ----------
				if (len(article_[1].split(" "))>1):
					try:
						receiver_ = normalize_counter(Counter([x.split('+')[0] for x in article_[0].split("@") if remove_position_ not in x]))
						
						sender_ = [x.split('^^')[1:3] for x in article_[1].split(" ") if "Different_Field" not in x and  int(article_[1].split("^^")[2]) <= (year+input_window)]

						sender_ = sum([normalize_counter(Counter([y.split('+')[0] for y in x[0].split("@") if (remove_position_ not in y)])) for x in sender_ if x!=[]],Counter())
						citation_list.append([year,receiver_,sender_])
					except:
						continue
				else:
					continue 

			citation_df = pd.DataFrame(citation_list,columns=["Year","Receiver","Sender"])

			citation_df = pd.DataFrame([(year, receiver_, k,v) for (year,receiver_,sender_) in citation_df.values for k, v in sender_.items()],columns=["Year","Receiver","Sender_ROR","Sender_Cite"])

			citation_df = pd.DataFrame([(year,k,v,sender_,Sender_cite) for (year,receiver_,sender_,Sender_cite) in citation_df.values for k, v in receiver_.items()],columns=["Year","Receiver_ROR","Receiver_Cite","Sender_ROR","Sender_Cite"])

			# N.B., Need 'dfply' -----------
			citation_df = (citation_df>>
				group_by(X.Year,X.Receiver_ROR,X.Sender_ROR)>>
				summarize(Citations = (X.Sender_Cite * X.Receiver_Cite).sum()))

			citation_df = citation_df.query("Receiver_ROR!='' & Sender_ROR!=''")
			citation_df["N_Citations_Received_Timeframe"] = input_window
			full_citation_df = pd.concat([full_citation_df, citation_df], ignore_index=True) #full_citation_df = full_citation_df.append(citation_df)

In [17]:
###############################
### Combine with ROR DataFrame
###############################

full_citation_with_ROR_df = pd.merge(full_citation_df,ror_df[["country.country_name","id","name"]].rename(columns={"country.country_name":"Country_Sender","name":"ROR_Sender_Name","id":"Sender_ROR"}),on=["Sender_ROR"],how="left")

full_citation_with_ROR_df = pd.merge(full_citation_with_ROR_df,ror_df[["country.country_name","id","name"]].rename(columns={"country.country_name":"Country_Receiver","name":"ROR_Receiver_Name","id":"Receiver_ROR"}),on=["Receiver_ROR"],how="left")

full_citation_with_ROR_df = full_citation_with_ROR_df.drop(columns=["Receiver_ROR","Sender_ROR"])


In [18]:
###############################
### Output final file to CSV
###############################

# N.B., need to update file name --------------
full_citation_with_ROR_df.to_csv("edge_list_ROR_to_ROR.csv")


In [1]:
full_citation_with_ROR_df.head()

NameError: name 'full_citation_with_ROR_df' is not defined

In [2]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install node2vec

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.0.1 requires triton==2.0.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 2.1.0 which is incompatible.
